# Auswertung des Fine-Tunings pro Anomalie-Art

In [40]:
import arrow
import numpy as np
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import joblib 
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset
from sklearn.metrics import confusion_matrix

from models.SimpleAutoEncoder import SimpleAutoEncoder

from utils.evalUtils import calc_cm_metrics
from utils.evalUtils import print_confusion_matrix

In [2]:
from matplotlib import rc
rc('text', usetex=True)

In [3]:
%run -i ./scripts/setConfigs.py

Set configs..


In [4]:
os.getcwd()

'/home/torge/dev/masterthesis_code/02_Experimente/03_Experimente'

In [5]:
%run -i ./scripts/EvalPreperations.py

Set configs..
Read the data..
Shape of normal data (X_sim): (105216, 17)
Shape of anormal data (X_test): (35040, 18)
Shape of drifted data (X_drifted): (35040, 18)
Shape of drifted anormal data (X_drifted,anormal): (35040, 19)
Save label..
Shape of anormal data (X_test): (35040, 17)
Shape of drifted data (X_drifted): (35040, 17)
Shape of drifted anormal data (X_drifted,anormal): (35040, 17)
Scale data..
Prepare data for PyTorch..


In [6]:
type(drifted_anormal_torch_tensor)

torch.Tensor

## Read Experimet Results from Phase I

In [7]:
os.chdir(os.path.join(exp_data_path, 'experiment'))
extension = 'csv'
result = glob.glob('*.{}'.format(extension))

In [8]:
result[0]

'20200408_EXPERIMENT__K_TRAIN_5_NUM_TRAIN_32_NUM_EVAL_32_NUM_ITER_100_OPTIMIZER_Adam_META_LR_0.1_ADAPTION_STEPS_15.csv'

In [9]:
df_tVP1_m1 = pd.read_csv(result[0])

In [10]:
for file in result[1:]:
    df = pd.read_csv(file)
    df_tVP1_m1 = df_tVP1_m1.append(df)

In [11]:
df_tVP1_m1.reset_index(inplace=True)

In [12]:
len(df_tVP1_m1)

97

In [13]:
if 'Unnamed: 0' in df_tVP1_m1.columns:
    print('Drop unnamed shit col...')
    df_tVP1_m1.drop('Unnamed: 0', axis=1,inplace=True)

Drop unnamed shit col...


In [14]:
df_tVP1_m1.head(1)

,index,optimizer,adaption_steps,meta_lr,num_iterations,k_train,num_train,num_eval,model_fn,logreg_fn,num_samples_x_test,num_samples_x_ano_drifted,TP_x_test,TN_x_test,FP_x_test,FN_x_test,TP_x_ano_drifted,TN_x_ano_drifted,FP_x_ano_drifted,FN_x_ano_drifted,Accuracy_x_test,Precision_x_test,Specifity_x_test,Sensitivity_x_test,F1_x_test,Accuracy_x_ano_drifted,Precision_x_ano_drifted,Specifity_x_ano_drifted,Sensitivity_x_ano_drifted,F1_x_ano_drifted
0,0,Adam,15,0.1,100,5,32,32,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,35040,35040,2407,32543,0,90,2417,29230,3313,80,99.743151,100.0,93.113019,96.395675,98.164763,90.316781,42.181501,92.362625,96.796155,58.757749


In [15]:
df_results = pd.DataFrame()

In [16]:
df_results['ano_labels'] = s_ano_labels_drifted_ano
df_results.reset_index(inplace=True, drop=True)

## Let every model predict on $X_{drifted,ano}$

In [17]:
for i, row in df_tVP1_m1.iterrows():
    print('Current Iteration: {} of {}'.format(i+1, len(df_tVP1_m1)))
    model = None
    model = SimpleAutoEncoder(num_inputs=17, val_lambda=42)
    logreg = None
    
    model_fn = row['model_fn']
    logreg_fn = row['logreg_fn']
    
    logreg = joblib.load(logreg_fn)
    model.load_state_dict(torch.load(model_fn))
    
    losses = []
    for val in drifted_anormal_torch_tensor:
        loss = model.calc_reconstruction_error(val)
        losses.append(loss.item())
    
    s_losses_anormal = pd.Series(losses)
    
    X = s_losses_anormal.to_numpy()
    X = X.reshape(-1, 1)
    
    predictions = []
    for val in X:
        val = val.reshape(1,-1)
        pred = logreg.predict(val)
        predictions.append(pred[0])

    col_name = 'preds_model_{}'.format(i)
    
    df_results[col_name] = predictions

Current Iteration: 1 of 97
Current Iteration: 2 of 97
Current Iteration: 3 of 97
Current Iteration: 4 of 97
Current Iteration: 5 of 97
Current Iteration: 6 of 97
Current Iteration: 7 of 97
Current Iteration: 8 of 97
Current Iteration: 9 of 97
Current Iteration: 10 of 97
Current Iteration: 11 of 97
Current Iteration: 12 of 97
Current Iteration: 13 of 97
Current Iteration: 14 of 97
Current Iteration: 15 of 97
Current Iteration: 16 of 97
Current Iteration: 17 of 97
Current Iteration: 18 of 97
Current Iteration: 19 of 97
Current Iteration: 20 of 97
Current Iteration: 21 of 97
Current Iteration: 22 of 97
Current Iteration: 23 of 97
Current Iteration: 24 of 97
Current Iteration: 25 of 97
Current Iteration: 26 of 97
Current Iteration: 27 of 97
Current Iteration: 28 of 97
Current Iteration: 29 of 97
Current Iteration: 30 of 97
Current Iteration: 31 of 97
Current Iteration: 32 of 97
Current Iteration: 33 of 97
Current Iteration: 34 of 97
Current Iteration: 35 of 97
Current Iteration: 36 of 97
C

In [ ]:
file_name = '{}_predictions_models_phase_1_on_x_drifted_ano.csv'.format(arrow.now().format('YYYYMMDD'))
full_fn = '/home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/exp_data/evaluation/{}'.format(file_name)
df_results.to_csv(full_fn, sep=';', index=False)

In [234]:
len(df_results)

35040

In [238]:
len(df_results.columns)

99

In [239]:
df_results.head()

,ano_labels,preds_model_0,preds_model_1,preds_model_2,preds_model_3,preds_model_4,preds_model_5,preds_model_6,preds_model_7,preds_model_8,preds_model_9,preds_model_10,preds_model_11,preds_model_12,preds_model_13,preds_model_14,preds_model_15,preds_model_16,preds_model_17,preds_model_18,preds_model_19,preds_model_20,preds_model_21,preds_model_22,preds_model_23,preds_model_24,preds_model_25,preds_model_26,preds_model_27,preds_model_28,preds_model_29,preds_model_30,preds_model_31,preds_model_32,preds_model_33,preds_model_34,preds_model_35,preds_model_36,preds_model_37,preds_model_38,preds_model_39,preds_model_40,preds_model_41,preds_model_42,preds_model_43,preds_model_44,preds_model_45,preds_model_46,preds_model_47,preds_model_48,preds_model_49,preds_model_50,preds_model_51,preds_model_52,preds_model_53,preds_model_54,preds_model_55,preds_model_56,preds_model_57,preds_model_58,preds_model_59,preds_model_60,preds_model_61,preds_model_62,preds_model_63,preds_model_64,preds_model_65,preds_model_66,preds_model_67,preds_model_68,preds_model_69,preds_model_70,preds_model_71,preds_model_72,preds_model_73,preds_model_74,preds_model_75,preds_model_76,preds_model_77,preds_model_78,preds_model_79,preds_model_80,preds_model_81,preds_model_82,preds_model_83,preds_model_84,preds_model_85,preds_model_86,preds_model_87,preds_model_88,preds_model_89,preds_model_90,preds_model_91,preds_model_92,preds_model_93,preds_model_94,preds_model_95,preds_model_96,binary_label
0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [237]:
df_results['binary_label'] = [1 if x > 0 else 0 for x in df_results['ano_labels']]

In [240]:
phase_1_general_kpis_per_model = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}

for col in df_results.columns:
    if col != 'ano_labels' and col != 'binary_label':
        cm = confusion_matrix(df_results['binary_label'], df_results[col])
        tn, fp, fn, tp  = cm.ravel()
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
        
        phase_1_general_kpis_per_model['acc'].append(accuracy)
        phase_1_general_kpis_per_model['prec'].append(precision)
        phase_1_general_kpis_per_model['spec'].append(specifity)
        phase_1_general_kpis_per_model['sen'].append(sensitivity)

In [241]:
df_phase_1_general_kpis_per_model = pd.DataFrame(phase_1_general_kpis_per_model)

In [242]:
df_phase_1_general_kpis_per_model.head()

,acc,prec,spec,sen
0,90.316781,42.181501,92.362625,96.796155
1,90.296804,42.110781,92.383059,96.515819
2,90.271119,42.019601,92.409345,96.155386
3,90.285388,42.070292,92.394740,96.355627
4,90.308219,42.151214,92.371382,96.676011


In [243]:
df_phase_1_general_kpis_per_model.describe()

,acc,prec,spec,sen
count,97.000000,97.000000,97.000000,97.000000
mean,90.254466,41.959457,92.426440,95.921704
std,0.063274,0.227448,0.064899,0.887908
min,89.917237,40.733453,92.336366,91.189427
25%,90.234018,41.887388,92.388899,95.634762
50%,90.268265,42.009452,92.412267,96.115338
75%,90.291096,42.090544,92.447340,96.435723
max,90.342466,42.272173,92.773035,97.156588


In [34]:
df_results_anos_0 = df_results[df_results['ano_labels'] == 0.0]
df_results_anos_1 = df_results[df_results['ano_labels'] == 1.0]
df_results_anos_2 = df_results[df_results['ano_labels'] == 2.0]
df_results_anos_3 = df_results[df_results['ano_labels'] == 3.0]

In [35]:
print('Anzahl Samples in Klasse 0: {}'.format(len(df_results_anos_0)))
print('Anzahl Samples in Klasse 1: {}'.format(len(df_results_anos_1)))
print('Anzahl Samples in Klasse 2: {}'.format(len(df_results_anos_2)))
print('Anzahl Samples in Klasse 3: {}'.format(len(df_results_anos_3)))

Anzahl Samples in Klasse 0: 32543
Anzahl Samples in Klasse 1: 20
Anzahl Samples in Klasse 2: 2433
Anzahl Samples in Klasse 3: 44


## Create KPIs for every model per Anomaly class

In [36]:
kpis_per_model_anos_0 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}
kpis_per_model_anos_1 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}
kpis_per_model_anos_2 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}
kpis_per_model_anos_3 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}

In [38]:
label_ano_class_0 = np.zeros(len(df_results_anos_0))
label_ano_class_1 = np.ones(len(df_results_anos_1))
label_ano_class_2 = np.ones(len(df_results_anos_2))
label_ano_class_3 = np.ones(len(df_results_anos_3))

In [41]:
for col in df_results_anos_0.columns:
    if col != 'ano_labels':
        cm = confusion_matrix(label_ano_class_0, df_results_anos_0[col])
        tn, fp, fn, tp  = cm.ravel()
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
        
        kpis_per_model_anos_0['acc'].append(accuracy)
        kpis_per_model_anos_0['prec'].append(precision)
        kpis_per_model_anos_0['spec'].append(specifity)
        kpis_per_model_anos_0['sen'].append(sensitivity)

In [42]:
df_kpis_per_model_anos_0 = pd.DataFrame(kpis_per_model_anos_0)

In [43]:
df_kpis_per_model_anos_0.head()

,acc,prec,spec,sen
0,89.819623,0.0,100.0,0.0
1,89.819623,0.0,100.0,0.0
2,89.819623,0.0,100.0,0.0
3,89.819623,0.0,100.0,0.0
4,89.819623,0.0,100.0,0.0


In [78]:
df_kpis_per_model_anos_0.describe()

,acc,prec,spec,sen
count,9.700000e+01,97.0,97.0,97.0
mean,8.981962e+01,0.0,100.0,0.0
std,1.428468e-14,0.0,0.0,0.0
min,8.981962e+01,0.0,100.0,0.0
25%,8.981962e+01,0.0,100.0,0.0
50%,8.981962e+01,0.0,100.0,0.0
75%,8.981962e+01,0.0,100.0,0.0
max,8.981962e+01,0.0,100.0,0.0


In [244]:
print(1.428468e-14)

1.428468e-14


In [44]:
len(df_kpis_per_model_anos_0)

97

In [45]:
for col in df_results_anos_1.columns:
    if col != 'ano_labels':
        cm = confusion_matrix(label_ano_class_1, df_results_anos_1[col])
        tn, fp, fn, tp  = cm.ravel()
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
        
        kpis_per_model_anos_1['acc'].append(accuracy)
        kpis_per_model_anos_1['prec'].append(precision)
        kpis_per_model_anos_1['spec'].append(specifity)
        kpis_per_model_anos_1['sen'].append(sensitivity)

In [46]:
df_kpis_per_model_anos_1 = pd.DataFrame(kpis_per_model_anos_1)

In [47]:
df_kpis_per_model_anos_1.head()

,acc,prec,spec,sen
0,20.0,100.0,0.0,20.0
1,15.0,100.0,0.0,15.0
2,15.0,100.0,0.0,15.0
3,15.0,100.0,0.0,15.0
4,15.0,100.0,0.0,15.0


In [79]:
df_kpis_per_model_anos_1.describe()

,acc,prec,spec,sen
count,97.000000,9.700000e+01,97.0,97.000000
mean,15.721649,1.000000e+02,0.0,15.721649
std,1.766248,2.943746e-08,0.0,1.766248
min,15.000000,1.000000e+02,0.0,15.000000
25%,15.000000,1.000000e+02,0.0,15.000000
50%,15.000000,1.000000e+02,0.0,15.000000
75%,15.000000,1.000000e+02,0.0,15.000000
max,20.000000,1.000000e+02,0.0,20.000000


In [48]:
for col in df_results_anos_2.columns:
    if col != 'ano_labels':
        cm = confusion_matrix(label_ano_class_2, df_results_anos_2[col])
        tn, fp, fn, tp  = cm.ravel()
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
        
        kpis_per_model_anos_2['acc'].append(accuracy)
        kpis_per_model_anos_2['prec'].append(precision)
        kpis_per_model_anos_2['spec'].append(specifity)
        kpis_per_model_anos_2['sen'].append(sensitivity)

In [49]:
df_kpis_per_model_anos_2 = pd.DataFrame(kpis_per_model_anos_2)

In [50]:
df_kpis_per_model_anos_2.head()

,acc,prec,spec,sen
0,98.684751,100.0,0.0,98.684751
1,98.438142,100.0,0.0,98.438142
2,98.027127,100.0,0.0,98.027127
3,98.273736,100.0,0.0,98.273736
4,98.602548,100.0,0.0,98.602548


In [80]:
df_kpis_per_model_anos_2.describe()

,acc,prec,spec,sen
count,97.000000,9.700000e+01,97.0,97.000000
mean,97.813992,1.000000e+02,0.0,97.813992
std,0.904999,3.975705e-12,0.0,0.904999
min,92.971640,1.000000e+02,0.0,92.971640
25%,97.533909,1.000000e+02,0.0,97.533909
50%,97.986025,1.000000e+02,0.0,97.986025
75%,98.355939,1.000000e+02,0.0,98.355939
max,99.013564,1.000000e+02,0.0,99.013564


In [51]:
for col in df_results_anos_3.columns:
    if col != 'ano_labels':
        cm = confusion_matrix(label_ano_class_3, df_results_anos_3[col])
        tn, fp, fn, tp  = cm.ravel()
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
        
        kpis_per_model_anos_3['acc'].append(accuracy)
        kpis_per_model_anos_3['prec'].append(precision)
        kpis_per_model_anos_3['spec'].append(specifity)
        kpis_per_model_anos_3['sen'].append(sensitivity)

In [52]:
df_kpis_per_model_anos_3 = pd.DataFrame(kpis_per_model_anos_3)

In [53]:
df_kpis_per_model_anos_3.head()

,acc,prec,spec,sen
0,27.272727,100.0,0.0,27.272727
1,27.272727,100.0,0.0,27.272727
2,29.545455,100.0,0.0,29.545455
3,27.272727,100.0,0.0,27.272727
4,27.272727,100.0,0.0,27.272727


In [81]:
df_kpis_per_model_anos_3.describe()

,acc,prec,spec,sen
count,97.000000,9.700000e+01,97.0,97.000000
mean,27.741331,1.000000e+02,0.0,27.741331
std,0.924243,2.606841e-09,0.0,0.924243
min,27.272727,1.000000e+02,0.0,27.272727
25%,27.272727,1.000000e+02,0.0,27.272727
50%,27.272727,1.000000e+02,0.0,27.272727
75%,27.272727,1.000000e+02,0.0,27.272727
max,29.545455,1.000000e+02,0.0,29.545455


In [82]:
arrow.now()

<Arrow [2020-04-11T18:03:12.737705+02:00]>

## Results from Phase II

In [83]:
os.getcwd()

'/home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/exp_data/experiment/fine_tuning'

In [56]:
os.chdir(os.path.join(exp_data_path, 'experiment', 'fine_tuning'))
extension = 'csv'
result = glob.glob('*.{}'.format(extension))

In [94]:
real_results = []

for res in result:
    if 'tVPII_M2' not in res:
        real_results.append(res)

In [95]:
df_tVP2_m1 = pd.read_csv(real_results[0], sep=';')

In [96]:
df_tVP2_m1.head()

,optimizer,fine_tune_classes,name_pretrained_model,k,fine_tune_iterations,lr,model_fn,pretrained_model_fn,logreg_fn,TP_x_test,TN_x_test,FP_x_test,FN_x_test,TP_x_drifted_ano,TN_x_drifted_ano,FP_x_drifted_ano,FN_x_drifted_ano,Accuracy_x_test,Precision_x_test,Specifity_x_test,Sensitivity_x_test,Accuracy_x_drifted_ano,Precision_x_drifted_ano,Specifity_x_drifted_ano,Sensitivity_x_drifted_ano
0,Adam,[2],3,20,64,0.001,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,2413,31468,1075,84,2412,28402,4141,85,96.692352,69.180046,92.878014,96.635963,87.939498,36.807569,92.172389,96.595915


In [97]:
for file in real_results[1:]:
    df = pd.read_csv(file, sep=';')
    df_tVP2_m1 = df_tVP2_m1.append(df)

In [98]:
df_tVP2_m1.reset_index(drop=True, inplace=True)

In [99]:
df_tVP2_m1.head(2)

,optimizer,fine_tune_classes,name_pretrained_model,k,fine_tune_iterations,lr,model_fn,pretrained_model_fn,logreg_fn,TP_x_test,TN_x_test,FP_x_test,FN_x_test,TP_x_drifted_ano,TN_x_drifted_ano,FP_x_drifted_ano,FN_x_drifted_ano,Accuracy_x_test,Precision_x_test,Specifity_x_test,Sensitivity_x_test,Accuracy_x_drifted_ano,Precision_x_drifted_ano,Specifity_x_drifted_ano,Sensitivity_x_drifted_ano
0,Adam,[2],3,20,64,0.001,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,2413,31468,1075,84,2412,28402,4141,85,96.692352,69.180046,92.878014,96.635963,87.939498,36.807569,92.172389,96.595915
1,SGD,"[1, 2, 3]",6,10,64,0.010,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,/home/torge/dev/masterthesis_code/02_Experimen...,2487,15098,17445,10,2487,13630,18913,10,50.185502,12.477423,85.857265,99.599519,45.996005,11.621495,84.569089,99.599519


In [100]:
len(df_tVP2_m1)

226

In [101]:
df_results_phase_2 = pd.DataFrame()

In [102]:
df_results_phase_2['ano_labels'] = s_ano_labels_drifted_ano
df_results_phase_2.reset_index(inplace=True, drop=True)

In [103]:
print('Started: {}'.format(arrow.now().format('HH:mm:ss')))
for i, row in df_tVP2_m1.iterrows():
    print('Current Iteration: {} of {}'.format(i+1, len(df_tVP2_m1)))
    model = None
    model = SimpleAutoEncoder(num_inputs=17, val_lambda=42)
    logreg = None
    
    model_fn = row['model_fn']
    logreg_fn = row['logreg_fn']
    
    logreg = joblib.load(logreg_fn)
    model.load_state_dict(torch.load(model_fn))
    
    losses = []
    for val in drifted_anormal_torch_tensor:
        loss = model.calc_reconstruction_error(val)
        losses.append(loss.item())
    
    s_losses_anormal = pd.Series(losses)
    
    X = s_losses_anormal.to_numpy()
    X = X.reshape(-1, 1)
    
    predictions = []
    for val in X:
        val = val.reshape(1,-1)
        pred = logreg.predict(val)
        predictions.append(pred[0])

    col_name = 'preds_model_{}'.format(i)
    
    df_results_phase_2[col_name] = predictions

print('Ended: {}'.format(arrow.now().format('HH:mm:ss')))

Started: 18:05:02
Current Iteration: 1 of 226
Current Iteration: 2 of 226
Current Iteration: 3 of 226
Current Iteration: 4 of 226
Current Iteration: 5 of 226
Current Iteration: 6 of 226
Current Iteration: 7 of 226
Current Iteration: 8 of 226
Current Iteration: 9 of 226
Current Iteration: 10 of 226
Current Iteration: 11 of 226
Current Iteration: 12 of 226
Current Iteration: 13 of 226
Current Iteration: 14 of 226
Current Iteration: 15 of 226
Current Iteration: 16 of 226
Current Iteration: 17 of 226
Current Iteration: 18 of 226
Current Iteration: 19 of 226
Current Iteration: 20 of 226
Current Iteration: 21 of 226
Current Iteration: 22 of 226
Current Iteration: 23 of 226
Current Iteration: 24 of 226
Current Iteration: 25 of 226
Current Iteration: 26 of 226
Current Iteration: 27 of 226
Current Iteration: 28 of 226
Current Iteration: 29 of 226
Current Iteration: 30 of 226
Current Iteration: 31 of 226
Current Iteration: 32 of 226
Current Iteration: 33 of 226
Current Iteration: 34 of 226
Curre

In [ ]:
file_name = '{}_predictions_models_phase_2_on_x_drifted_ano.csv'.format(arrow.now().format('YYYYMMDD'))
full_fn = '/home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/exp_data/evaluation/{}'.format(file_name)
df_results_phase_2.to_csv(full_fn, sep=';', index=False)

In [245]:
len(df_results_phase_2)

35040

In [196]:
len(df_results_phase_2.columns)

228

In [250]:
print(df_results_phase_2['binary_label'].sum())
print(len(df_results_phase_2))

2497
35040
32543


In [197]:
df_results_phase_2.head()

,ano_labels,preds_model_0,preds_model_1,preds_model_2,preds_model_3,preds_model_4,preds_model_5,preds_model_6,preds_model_7,preds_model_8,preds_model_9,preds_model_10,preds_model_11,preds_model_12,preds_model_13,preds_model_14,preds_model_15,preds_model_16,preds_model_17,preds_model_18,preds_model_19,preds_model_20,preds_model_21,preds_model_22,preds_model_23,preds_model_24,preds_model_25,preds_model_26,preds_model_27,preds_model_28,preds_model_29,preds_model_30,preds_model_31,preds_model_32,preds_model_33,preds_model_34,preds_model_35,preds_model_36,preds_model_37,preds_model_38,preds_model_39,preds_model_40,preds_model_41,preds_model_42,preds_model_43,preds_model_44,preds_model_45,preds_model_46,preds_model_47,preds_model_48,preds_model_49,preds_model_50,preds_model_51,preds_model_52,preds_model_53,preds_model_54,preds_model_55,preds_model_56,preds_model_57,preds_model_58,preds_model_59,preds_model_60,preds_model_61,preds_model_62,preds_model_63,preds_model_64,preds_model_65,preds_model_66,preds_model_67,preds_model_68,preds_model_69,preds_model_70,preds_model_71,preds_model_72,preds_model_73,preds_model_74,preds_model_75,preds_model_76,preds_model_77,preds_model_78,preds_model_79,preds_model_80,preds_model_81,preds_model_82,preds_model_83,preds_model_84,preds_model_85,preds_model_86,preds_model_87,preds_model_88,preds_model_89,preds_model_90,preds_model_91,preds_model_92,preds_model_93,preds_model_94,preds_model_95,preds_model_96,preds_model_97,preds_model_98,preds_model_99,preds_model_100,preds_model_101,preds_model_102,preds_model_103,preds_model_104,preds_model_105,preds_model_106,preds_model_107,preds_model_108,preds_model_109,preds_model_110,preds_model_111,preds_model_112,preds_model_113,preds_model_114,preds_model_115,preds_model_116,preds_model_117,preds_model_118,preds_model_119,preds_model_120,preds_model_121,preds_model_122,preds_model_123,preds_model_124,preds_model_125,preds_model_126,preds_model_127,preds_model_128,preds_model_129,preds_model_130,preds_model_131,preds_model_132,preds_model_133,preds_model_134,preds_model_135,preds_model_136,preds_model_137,preds_model_138,preds_model_139,preds_model_140,preds_model_141,preds_model_142,preds_model_143,preds_model_144,preds_model_145,preds_model_146,preds_model_147,preds_model_148,preds_model_149,preds_model_150,preds_model_151,preds_model_152,preds_model_153,preds_model_154,preds_model_155,preds_model_156,preds_model_157,preds_model_158,preds_model_159,preds_model_160,preds_model_161,preds_model_162,preds_model_163,preds_model_164,preds_model_165,preds_model_166,preds_model_167,preds_model_168,preds_model_169,preds_model_170,preds_model_171,preds_model_172,preds_model_173,preds_model_174,preds_model_175,preds_model_176,preds_model_177,preds_model_178,preds_model_179,preds_model_180,preds_model_181,preds_model_182,preds_model_183,preds_model_184,preds_model_185,preds_model_186,preds_model_187,preds_model_188,preds_model_189,preds_model_190,preds_model_191,preds_model_192,preds_model_193,preds_model_194,preds_model_195,preds_model_196,preds_model_197,preds_model_198,preds_model_199,preds_model_200,preds_model_201,preds_model_202,preds_model_203,preds_model_204,preds_model_205,preds_model_206,preds_model_207,preds_model_208,preds_model_209,preds_model_210,preds_model_211,preds_model_212,preds_model_213,preds_model_214,preds_model_215,preds_model_216,preds_model_217,preds_model_218,preds_model_219,preds_model_220,preds_model_221,preds_model_222,preds_model_223,preds_model_224,preds_model_225,binary_label
0,0.0,0,1,0,0,1,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0,1,1,0,0,1,1,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1,1,0,1,0,0,0
1,0.0,0,1

In [198]:
df_results_phase_2['binary_label'] = [1 if x > 0 else 0 for x in df_results_phase_2['ano_labels']]

In [199]:
phase_2_general_kpis_per_model = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}

In [200]:
for col in df_results_phase_2.columns:
    if col != 'ano_labels' and col != 'binary_label':
        cm = confusion_matrix(df_results_phase_2['binary_label'], df_results_phase_2[col])
        tn, fp, fn, tp  = cm.ravel()
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
        
        phase_2_general_kpis_per_model['acc'].append(accuracy)
        phase_2_general_kpis_per_model['prec'].append(precision)
        phase_2_general_kpis_per_model['spec'].append(specifity)
        phase_2_general_kpis_per_model['sen'].append(sensitivity)

In [201]:
df_phase_2_general_kpis_per_model = pd.DataFrame(phase_2_general_kpis_per_model)

In [202]:
df_phase_2_general_kpis_per_model.describe()

,acc,prec,spec,sen
count,226.000000,226.000000,226.000000,226.000000
mean,55.894401,17.999863,82.942866,98.426608
std,23.233713,10.433545,13.896941,1.648265
min,7.126142,7.126142,0.000000,91.389668
25%,40.284675,10.625956,82.456147,97.797357
50%,56.482591,13.933007,87.555234,98.758510
75%,75.541524,22.352302,90.725261,99.559471
max,90.368151,42.362561,92.758314,100.000000


## Analyse pro Anomalie Art für Phase II

In [203]:
df_results_phase_2_anos_0 = df_results_phase_2[df_results_phase_2['ano_labels'] == 0.0]
df_results_phase_2_anos_1 = df_results_phase_2[df_results_phase_2['ano_labels'] == 1.0]
df_results_phase_2_anos_2 = df_results_phase_2[df_results_phase_2['ano_labels'] == 2.0]
df_results_phase_2_anos_3 = df_results_phase_2[df_results_phase_2['ano_labels'] == 3.0]

In [204]:
print('Anzahl Samples in Klasse 0: {}'.format(len(df_results_phase_2_anos_0)))
print('Anzahl Samples in Klasse 1: {}'.format(len(df_results_phase_2_anos_1)))
print('Anzahl Samples in Klasse 2: {}'.format(len(df_results_phase_2_anos_2)))
print('Anzahl Samples in Klasse 3: {}'.format(len(df_results_phase_2_anos_3)))

Anzahl Samples in Klasse 0: 32543
Anzahl Samples in Klasse 1: 20
Anzahl Samples in Klasse 2: 2433
Anzahl Samples in Klasse 3: 44


In [216]:
phase_2_kpis_per_model_anos_0 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}
phase_2_kpis_per_model_anos_1 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}
phase_2_kpis_per_model_anos_2 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}
phase_2_kpis_per_model_anos_3 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}

In [217]:
phase_2_label_ano_class_0 = np.zeros(len(df_results_phase_2_anos_0))
phase_2_label_ano_class_1 = np.ones(len(df_results_phase_2_anos_1))
phase_2_label_ano_class_2 = np.ones(len(df_results_phase_2_anos_2))
phase_2_label_ano_class_3 = np.ones(len(df_results_phase_2_anos_3))

### Kennzahlen Anomaly Klasse 0 nach Phase II Modell 1

In [218]:
for col in df_results_phase_2_anos_0.columns:
    if col != 'ano_labels' and col != 'binary_label':
        cm = confusion_matrix(phase_2_label_ano_class_0, df_results_phase_2_anos_0[col])
        try:
            tn, fp, fn, tp  = cm.ravel()
        except:
            print('Cant ravel CM : {}'.format(col))
            if df_results_phase_2_anos_0[col].all() == 1:
                tp = 0
                tn = 0
                fp = cm[0][0]
                fn = 0
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
        
        phase_2_kpis_per_model_anos_0['acc'].append(accuracy)
        phase_2_kpis_per_model_anos_0['prec'].append(precision)
        phase_2_kpis_per_model_anos_0['spec'].append(specifity)
        phase_2_kpis_per_model_anos_0['sen'].append(sensitivity)

In [219]:
df_phase_2_kpis_per_model_anos_0 = pd.DataFrame(phase_2_kpis_per_model_anos_0)

In [220]:
df_phase_2_kpis_per_model_anos_0.head()

,acc,prec,spec,sen
0,87.275297,0.0,100.0,0.0
1,41.883047,0.0,100.0,0.0
2,79.442584,0.0,100.0,0.0
3,84.936853,0.0,100.0,0.0
4,37.882187,0.0,100.0,0.0


In [221]:
df_phase_2_kpis_per_model_anos_0.describe()

,acc,prec,spec,sen
count,226.000000,226.0,226.000000,226.0
mean,52.630937,0.0,99.115044,0.0
std,25.107936,0.0,9.386280,0.0
min,0.000000,0.0,0.000000,0.0
25%,35.735028,0.0,100.000000,0.0
50%,53.248010,0.0,100.000000,0.0
75%,73.793903,0.0,100.000000,0.0
max,89.819623,0.0,100.000000,0.0


### Kennzahlen Anomaly Klasse 1 nach Phase II Modell 1

In [222]:
for col in df_results_phase_2_anos_1.columns:
    if col != 'ano_labels' and col != 'binary_label':
        cm = confusion_matrix(phase_2_label_ano_class_1, df_results_phase_2_anos_1[col])
        try:
            tn, fp, fn, tp  = cm.ravel()
        except:
            print('Cant ravel CM : {}'.format(col))
            if df_results_phase_2_anos_1[col].all() == 1:
                tp = cm[0][0]
                tn = 0
                fp = 0
                fn = 0
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
        
        phase_2_kpis_per_model_anos_1['acc'].append(accuracy)
        phase_2_kpis_per_model_anos_1['prec'].append(precision)
        phase_2_kpis_per_model_anos_1['spec'].append(specifity)
        phase_2_kpis_per_model_anos_1['sen'].append(sensitivity)

Cant ravel CM : preds_model_6
Cant ravel CM : preds_model_31
Cant ravel CM : preds_model_91
Cant ravel CM : preds_model_99
Cant ravel CM : preds_model_115


In [223]:
df_phase_2_kpis_per_model_anos_1 = pd.DataFrame(phase_2_kpis_per_model_anos_1)

In [224]:
df_phase_2_kpis_per_model_anos_1.head()

,acc,prec,spec,sen
0,25.0,100.0,0.0,25.0
1,85.0,100.0,0.0,85.0
2,30.0,100.0,0.0,30.0
3,25.0,100.0,0.0,25.0
4,75.0,100.0,0.0,75.0


In [225]:
df_phase_2_kpis_per_model_anos_1.describe()

,acc,prec,spec,sen
count,226.000000,2.260000e+02,226.0,226.000000
mean,57.212389,1.000000e+02,0.0,57.212389
std,24.128711,7.173107e-08,0.0,24.128711
min,15.000000,1.000000e+02,0.0,15.000000
25%,40.000000,1.000000e+02,0.0,40.000000
50%,55.000000,1.000000e+02,0.0,55.000000
75%,80.000000,1.000000e+02,0.0,80.000000
max,100.000000,1.000000e+02,0.0,100.000000


### Kennzahlen Anomaly Klasse 2 nach Phase II Modell 1

In [226]:
for col in df_results_phase_2_anos_2.columns:
    if col != 'ano_labels' and col != 'binary_label':
        cm = confusion_matrix(phase_2_label_ano_class_2, df_results_phase_2_anos_2[col])
        try:
            tn, fp, fn, tp  = cm.ravel()
        except:
            print('Cant ravel CM : {}'.format(col))
            if df_results_phase_2_anos_2[col].all() == 1:
                tp = cm[0][0]
                tn = 0
                fp = 0
                fn = 0
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
        
        phase_2_kpis_per_model_anos_2['acc'].append(accuracy)
        phase_2_kpis_per_model_anos_2['prec'].append(precision)
        phase_2_kpis_per_model_anos_2['spec'].append(specifity)
        phase_2_kpis_per_model_anos_2['sen'].append(sensitivity)

Cant ravel CM : preds_model_5
Cant ravel CM : preds_model_6
Cant ravel CM : preds_model_8
Cant ravel CM : preds_model_10
Cant ravel CM : preds_model_25
Cant ravel CM : preds_model_27
Cant ravel CM : preds_model_29
Cant ravel CM : preds_model_31
Cant ravel CM : preds_model_42
Cant ravel CM : preds_model_46
Cant ravel CM : preds_model_50
Cant ravel CM : preds_model_57
Cant ravel CM : preds_model_79
Cant ravel CM : preds_model_91
Cant ravel CM : preds_model_99
Cant ravel CM : preds_model_114
Cant ravel CM : preds_model_131
Cant ravel CM : preds_model_136
Cant ravel CM : preds_model_137
Cant ravel CM : preds_model_141
Cant ravel CM : preds_model_148
Cant ravel CM : preds_model_152
Cant ravel CM : preds_model_156
Cant ravel CM : preds_model_161
Cant ravel CM : preds_model_162
Cant ravel CM : preds_model_166
Cant ravel CM : preds_model_169
Cant ravel CM : preds_model_170
Cant ravel CM : preds_model_174
Cant ravel CM : preds_model_178
Cant ravel CM : preds_model_180
Cant ravel CM : preds_mode

In [227]:
df_phase_2_kpis_per_model_anos_2 = pd.DataFrame(phase_2_kpis_per_model_anos_2)

In [228]:
df_phase_2_kpis_per_model_anos_2.head()

,acc,prec,spec,sen
0,98.438142,100.0,0.0,98.438142
1,99.958898,100.0,0.0,99.958898
2,99.753391,100.0,0.0,99.753391
3,99.013564,100.0,0.0,99.013564
4,99.876695,100.0,0.0,99.876695


In [229]:
df_phase_2_kpis_per_model_anos_2.describe()

,acc,prec,spec,sen
count,226.000000,2.260000e+02,226.0,226.000000
mean,99.159419,1.000000e+02,0.0,99.159419
std,1.292509,5.627263e-12,0.0,1.292509
min,93.177148,1.000000e+02,0.0,93.177148
25%,98.859433,1.000000e+02,0.0,98.859433
50%,99.527333,1.000000e+02,0.0,99.527333
75%,99.917797,1.000000e+02,0.0,99.917797
max,100.000000,1.000000e+02,0.0,100.000000


### Kennzahlen Anomaly Klasse 3 nach Phase II Modell 1

In [230]:
for col in df_results_phase_2_anos_3.columns:
    if col != 'ano_labels' and col !='binary_label':
        cm = confusion_matrix(phase_2_label_ano_class_3, df_results_phase_2_anos_3[col])
        try:
            tn, fp, fn, tp  = cm.ravel()
        except:
            print('Cant ravel CM : {}'.format(col))
            if df_results_phase_2_anos_3[col].all() == 1:
                tp = cm[0][0]
                tn = 0
                fp = 0
                fn = 0
                
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
                
        phase_2_kpis_per_model_anos_3['acc'].append(accuracy)
        phase_2_kpis_per_model_anos_3['prec'].append(precision)
        phase_2_kpis_per_model_anos_3['spec'].append(specifity)
        phase_2_kpis_per_model_anos_3['sen'].append(sensitivity)

Cant ravel CM : preds_model_6
Cant ravel CM : preds_model_31
Cant ravel CM : preds_model_45
Cant ravel CM : preds_model_46
Cant ravel CM : preds_model_69
Cant ravel CM : preds_model_89
Cant ravel CM : preds_model_91
Cant ravel CM : preds_model_99
Cant ravel CM : preds_model_115
Cant ravel CM : preds_model_142
Cant ravel CM : preds_model_148
Cant ravel CM : preds_model_201
Cant ravel CM : preds_model_202
Cant ravel CM : preds_model_208
Cant ravel CM : preds_model_221


In [231]:
df_phase_2_kpis_per_model_anos_3 = pd.DataFrame(phase_2_kpis_per_model_anos_3)

In [232]:
df_phase_2_kpis_per_model_anos_3.head()

,acc,prec,spec,sen
0,27.272727,100.0,0.0,27.272727
1,86.363636,100.0,0.0,86.363636
2,75.000000,100.0,0.0,75.000000
3,61.363636,100.0,0.0,61.363636
4,90.909091,100.0,0.0,90.909091


In [233]:
df_phase_2_kpis_per_model_anos_3.describe()

,acc,prec,spec,sen
count,226.000000,2.260000e+02,226.0,226.000000
mean,76.639179,1.000000e+02,0.0,76.639179
std,19.316511,1.434403e-08,0.0,19.316511
min,27.272727,1.000000e+02,0.0,27.272727
25%,65.909091,1.000000e+02,0.0,65.909091
50%,81.818182,1.000000e+02,0.0,81.818182
75%,90.909091,1.000000e+02,0.0,90.909091
max,100.000000,1.000000e+02,0.0,100.000000


# Veränderung der Modellperformance pro Concept Drift Event Art

In [252]:
df_results_phase_1_concept_drift = df_results.copy()

In [264]:
df_results_phase_1_concept_drift.head(1)

,ano_labels,preds_model_0,preds_model_1,preds_model_2,preds_model_3,preds_model_4,preds_model_5,preds_model_6,preds_model_7,preds_model_8,preds_model_9,preds_model_10,preds_model_11,preds_model_12,preds_model_13,preds_model_14,preds_model_15,preds_model_16,preds_model_17,preds_model_18,preds_model_19,preds_model_20,preds_model_21,preds_model_22,preds_model_23,preds_model_24,preds_model_25,preds_model_26,preds_model_27,preds_model_28,preds_model_29,preds_model_30,preds_model_31,preds_model_32,preds_model_33,preds_model_34,preds_model_35,preds_model_36,preds_model_37,preds_model_38,preds_model_39,preds_model_40,preds_model_41,preds_model_42,preds_model_43,preds_model_44,preds_model_45,preds_model_46,preds_model_47,preds_model_48,preds_model_49,preds_model_50,preds_model_51,preds_model_52,preds_model_53,preds_model_54,preds_model_55,preds_model_56,preds_model_57,preds_model_58,preds_model_59,preds_model_60,preds_model_61,preds_model_62,preds_model_63,preds_model_64,preds_model_65,preds_model_66,preds_model_67,preds_model_68,preds_model_69,preds_model_70,preds_model_71,preds_model_72,preds_model_73,preds_model_74,preds_model_75,preds_model_76,preds_model_77,preds_model_78,preds_model_79,preds_model_80,preds_model_81,preds_model_82,preds_model_83,preds_model_84,preds_model_85,preds_model_86,preds_model_87,preds_model_88,preds_model_89,preds_model_90,preds_model_91,preds_model_92,preds_model_93,preds_model_94,preds_model_95,preds_model_96,binary_label,cd_label
0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [259]:
s_drift_labls_copy = s_drift_labels_drifted_ano.copy()

In [260]:
s_drift_labls_copy.reset_index(drop=True, inplace=True)

In [263]:
df_results_phase_1_concept_drift['cd_label'] = s_drift_labls_copy

In [266]:
df_res_phase_1_cd_0 = df_results_phase_1_concept_drift[df_results_phase_1_concept_drift['cd_label'] == 0.0]
df_res_phase_1_cd_1 = df_results_phase_1_concept_drift[df_results_phase_1_concept_drift['cd_label'] == 1.0]
df_res_phase_1_cd_2 = df_results_phase_1_concept_drift[df_results_phase_1_concept_drift['cd_label'] == 2.0]
df_res_phase_1_cd_3 = df_results_phase_1_concept_drift[df_results_phase_1_concept_drift['cd_label'] == 3.0]

In [268]:
print('Anzahl CD Event 0: {}'.format(len(df_res_phase_1_cd_0)))
print('Anzahl CD Event 1: {}'.format(len(df_res_phase_1_cd_1)))
print('Anzahl CD Event 2: {}'.format(len(df_res_phase_1_cd_2)))
print('Anzahl CD Event 3: {}'.format(len(df_res_phase_1_cd_3)))

Anzahl CD Event 0: 1112
Anzahl CD Event 1: 3132
Anzahl CD Event 2: 17855
Anzahl CD Event 3: 12941


In [271]:
lab_res_phase_1_cd_0 = df_res_phase_1_cd_0['binary_label']
lab_res_phase_1_cd_1 = df_res_phase_1_cd_1['binary_label']
lab_res_phase_1_cd_2 = df_res_phase_1_cd_2['binary_label']
lab_res_phase_1_cd_3 = df_res_phase_1_cd_3['binary_label']

## Concept Drivt Event 0

In [273]:
kpis_phase_1_cd_0 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}

for col in df_res_phase_1_cd_0.columns:
    if col != 'ano_labels' and col !='binary_label' and col !='cd_label':
        cm = confusion_matrix(lab_res_phase_1_cd_0, df_res_phase_1_cd_0[col])
        try:
            tn, fp, fn, tp  = cm.ravel()
        except:
            print('Cant ravel CM : {}'.format(col))
            #if df_results_phase_2_anos_3[col].all() == 1:
             #   tp = cm[0][0]
              #  tn = 0
               # fp = 0
                #fn = 0
                
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
                
        kpis_phase_1_cd_0['acc'].append(accuracy)
        kpis_phase_1_cd_0['prec'].append(precision)
        kpis_phase_1_cd_0['spec'].append(specifity)
        kpis_phase_1_cd_0['sen'].append(sensitivity)

In [274]:
df_kpis_phase_1_cd_0 = pd.DataFrame(kpis_phase_1_cd_0)

In [275]:
df_kpis_phase_1_cd_0.describe()

,acc,prec,spec,sen
count,97.000000,9.700000e+01,97.000000,97.000000
mean,98.033635,1.000000e+02,99.621050,15.900079
std,0.162964,1.112047e-07,0.165169,6.969858
min,97.841727,1.000000e+02,98.997265,7.692308
25%,97.931655,1.000000e+02,99.541705,11.538462
50%,98.021583,1.000000e+02,99.633028,15.384615
75%,98.111511,1.000000e+02,99.724518,19.230769
max,98.651079,1.000000e+02,99.816176,42.307692


## Concept Drift Event 1

In [276]:
kpis_phase_1_cd_1 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}

for col in df_res_phase_1_cd_0.columns:
    if col != 'ano_labels' and col !='binary_label' and col !='cd_label':
        cm = confusion_matrix(lab_res_phase_1_cd_1, df_res_phase_1_cd_1[col])
        try:
            tn, fp, fn, tp  = cm.ravel()
        except:
            print('Cant ravel CM : {}'.format(col))
            #if df_results_phase_2_anos_3[col].all() == 1:
             #   tp = cm[0][0]
              #  tn = 0
               # fp = 0
                #fn = 0
                
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
                
        kpis_phase_1_cd_1['acc'].append(accuracy)
        kpis_phase_1_cd_1['prec'].append(precision)
        kpis_phase_1_cd_1['spec'].append(specifity)
        kpis_phase_1_cd_1['sen'].append(sensitivity)

In [277]:
df_kpis_phase_1_cd_1 = pd.DataFrame(kpis_phase_1_cd_1)

In [278]:
df_kpis_phase_1_cd_1.describe()

,acc,prec,spec,sen
count,9.700000e+01,9.700000e+01,97.0,97.0
mean,3.512133e-01,3.512133e-01,0.0,100.0
std,5.579952e-17,5.579952e-17,0.0,0.0
min,3.512133e-01,3.512133e-01,0.0,100.0
25%,3.512133e-01,3.512133e-01,0.0,100.0
50%,3.512133e-01,3.512133e-01,0.0,100.0
75%,3.512133e-01,3.512133e-01,0.0,100.0
max,3.512133e-01,3.512133e-01,0.0,100.0


## Concept Drift Event 2

In [279]:
kpis_phase_1_cd_2 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}

for col in df_res_phase_1_cd_0.columns:
    if col != 'ano_labels' and col !='binary_label' and col !='cd_label':
        cm = confusion_matrix(lab_res_phase_1_cd_2, df_res_phase_1_cd_2[col])
        try:
            tn, fp, fn, tp  = cm.ravel()
        except:
            print('Cant ravel CM : {}'.format(col))
            #if df_results_phase_2_anos_3[col].all() == 1:
             #   tp = cm[0][0]
              #  tn = 0
               # fp = 0
                #fn = 0
                
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
                
        kpis_phase_1_cd_2['acc'].append(accuracy)
        kpis_phase_1_cd_2['prec'].append(precision)
        kpis_phase_1_cd_2['spec'].append(specifity)
        kpis_phase_1_cd_2['sen'].append(sensitivity)

In [280]:
df_kpis_phase_1_cd_2 = pd.DataFrame(kpis_phase_1_cd_2)

In [281]:
df_kpis_phase_1_cd_2.describe()

,acc,prec,spec,sen
count,97.000000,97.000000,97.000000,97.000000
mean,98.814159,91.550175,88.209302,99.060383
std,0.047609,0.031785,0.042530,0.404792
min,98.599832,91.405551,88.180633,97.238095
25%,98.773453,91.523179,88.180633,98.714286
50%,98.846262,91.571554,88.180633,99.333333
75%,98.846262,91.571554,88.245634,99.333333
max,98.846262,91.571554,88.401022,99.333333


## Concept Drift Event 3

In [282]:
kpis_phase_1_cd_3 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}

for col in df_res_phase_1_cd_0.columns:
    if col != 'ano_labels' and col !='binary_label' and col !='cd_label':
        cm = confusion_matrix(lab_res_phase_1_cd_3, df_res_phase_1_cd_3[col])
        try:
            tn, fp, fn, tp  = cm.ravel()
        except:
            print('Cant ravel CM : {}'.format(col))
            #if df_results_phase_2_anos_3[col].all() == 1:
             #   tp = cm[0][0]
              #  tn = 0
               # fp = 0
                #fn = 0
                
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
                
        kpis_phase_1_cd_3['acc'].append(accuracy)
        kpis_phase_1_cd_3['prec'].append(precision)
        kpis_phase_1_cd_3['spec'].append(specifity)
        kpis_phase_1_cd_3['sen'].append(sensitivity)

In [283]:
df_kpis_phase_1_cd_3 = pd.DataFrame(kpis_phase_1_cd_3)

In [284]:
df_kpis_phase_1_cd_3.describe()

,acc,prec,spec,sen
count,97.000000,9.700000e+01,97.000000,97.000000
mean,99.534525,1.000000e+02,97.672937,83.267468
std,0.124014,2.037814e-10,0.121985,4.457946
min,98.933622,1.000000e+02,97.489345,61.666667
25%,99.497720,1.000000e+02,97.602793,81.944444
50%,99.551812,1.000000e+02,97.655826,83.888889
75%,99.605904,1.000000e+02,97.708916,85.833333
max,99.721814,1.000000e+02,98.266031,90.000000


## Auswertung Kennzahlen pro Concept Drift Event Art nach Phase II

In [286]:
df_results_phase_2_concept_drift = df_results_phase_2.copy()

In [288]:
df_results_phase_2_concept_drift['cd_label'] = s_drift_labls_copy

In [289]:
df_results_phase_2_concept_drift.head(1)

,ano_labels,preds_model_0,preds_model_1,preds_model_2,preds_model_3,preds_model_4,preds_model_5,preds_model_6,preds_model_7,preds_model_8,preds_model_9,preds_model_10,preds_model_11,preds_model_12,preds_model_13,preds_model_14,preds_model_15,preds_model_16,preds_model_17,preds_model_18,preds_model_19,preds_model_20,preds_model_21,preds_model_22,preds_model_23,preds_model_24,preds_model_25,preds_model_26,preds_model_27,preds_model_28,preds_model_29,preds_model_30,preds_model_31,preds_model_32,preds_model_33,preds_model_34,preds_model_35,preds_model_36,preds_model_37,preds_model_38,preds_model_39,preds_model_40,preds_model_41,preds_model_42,preds_model_43,preds_model_44,preds_model_45,preds_model_46,preds_model_47,preds_model_48,preds_model_49,preds_model_50,preds_model_51,preds_model_52,preds_model_53,preds_model_54,preds_model_55,preds_model_56,preds_model_57,preds_model_58,preds_model_59,preds_model_60,preds_model_61,preds_model_62,preds_model_63,preds_model_64,preds_model_65,preds_model_66,preds_model_67,preds_model_68,preds_model_69,preds_model_70,preds_model_71,preds_model_72,preds_model_73,preds_model_74,preds_model_75,preds_model_76,preds_model_77,preds_model_78,preds_model_79,preds_model_80,preds_model_81,preds_model_82,preds_model_83,preds_model_84,preds_model_85,preds_model_86,preds_model_87,preds_model_88,preds_model_89,preds_model_90,preds_model_91,preds_model_92,preds_model_93,preds_model_94,preds_model_95,preds_model_96,preds_model_97,preds_model_98,preds_model_99,preds_model_100,preds_model_101,preds_model_102,preds_model_103,preds_model_104,preds_model_105,preds_model_106,preds_model_107,preds_model_108,preds_model_109,preds_model_110,preds_model_111,preds_model_112,preds_model_113,preds_model_114,preds_model_115,preds_model_116,preds_model_117,preds_model_118,preds_model_119,preds_model_120,preds_model_121,preds_model_122,preds_model_123,preds_model_124,preds_model_125,preds_model_126,preds_model_127,preds_model_128,preds_model_129,preds_model_130,preds_model_131,preds_model_132,preds_model_133,preds_model_134,preds_model_135,preds_model_136,preds_model_137,preds_model_138,preds_model_139,preds_model_140,preds_model_141,preds_model_142,preds_model_143,preds_model_144,preds_model_145,preds_model_146,preds_model_147,preds_model_148,preds_model_149,preds_model_150,preds_model_151,preds_model_152,preds_model_153,preds_model_154,preds_model_155,preds_model_156,preds_model_157,preds_model_158,preds_model_159,preds_model_160,preds_model_161,preds_model_162,preds_model_163,preds_model_164,preds_model_165,preds_model_166,preds_model_167,preds_model_168,preds_model_169,preds_model_170,preds_model_171,preds_model_172,preds_model_173,preds_model_174,preds_model_175,preds_model_176,preds_model_177,preds_model_178,preds_model_179,preds_model_180,preds_model_181,preds_model_182,preds_model_183,preds_model_184,preds_model_185,preds_model_186,preds_model_187,preds_model_188,preds_model_189,preds_model_190,preds_model_191,preds_model_192,preds_model_193,preds_model_194,preds_model_195,preds_model_196,preds_model_197,preds_model_198,preds_model_199,preds_model_200,preds_model_201,preds_model_202,preds_model_203,preds_model_204,preds_model_205,preds_model_206,preds_model_207,preds_model_208,preds_model_209,preds_model_210,preds_model_211,preds_model_212,preds_model_213,preds_model_214,preds_model_215,preds_model_216,preds_model_217,preds_model_218,preds_model_219,preds_model_220,preds_model_221,preds_model_222,preds_model_223,preds_model_224,preds_model_225,binary_label,cd_label
0,0.0,0,1,0,0,1,1,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,1,0,0,1,1,0,0,1,1,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1,1,0,1,0,0,0,

In [290]:
df_res_phase_2_cd_0 = df_results_phase_2_concept_drift[df_results_phase_2_concept_drift['cd_label'] == 0.0]
df_res_phase_2_cd_1 = df_results_phase_2_concept_drift[df_results_phase_2_concept_drift['cd_label'] == 1.0]
df_res_phase_2_cd_2 = df_results_phase_2_concept_drift[df_results_phase_2_concept_drift['cd_label'] == 2.0]
df_res_phase_2_cd_3 = df_results_phase_2_concept_drift[df_results_phase_2_concept_drift['cd_label'] == 3.0]

In [291]:
print('Anzahl CD Event 0: {}'.format(len(df_res_phase_2_cd_0)))
print('Anzahl CD Event 1: {}'.format(len(df_res_phase_2_cd_1)))
print('Anzahl CD Event 2: {}'.format(len(df_res_phase_2_cd_2)))
print('Anzahl CD Event 3: {}'.format(len(df_res_phase_2_cd_3)))

Anzahl CD Event 0: 1112
Anzahl CD Event 1: 3132
Anzahl CD Event 2: 17855
Anzahl CD Event 3: 12941


In [293]:
lab_res_phase_2_cd_0 = df_res_phase_2_cd_0['binary_label']
lab_res_phase_2_cd_1 = df_res_phase_2_cd_1['binary_label']
lab_res_phase_2_cd_2 = df_res_phase_2_cd_2['binary_label']
lab_res_phase_2_cd_3 = df_res_phase_2_cd_3['binary_label']

### Concept Drift Event Art 0

In [294]:
kpis_phase_2_cd_0 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}

for col in df_res_phase_2_cd_0.columns:
    if col != 'ano_labels' and col !='binary_label' and col !='cd_label':
        cm = confusion_matrix(lab_res_phase_2_cd_0, df_res_phase_2_cd_0[col])
        try:
            tn, fp, fn, tp  = cm.ravel()
        except:
            print('Cant ravel CM : {}'.format(col))
            #if df_results_phase_2_anos_3[col].all() == 1:
             #   tp = cm[0][0]
              #  tn = 0
               # fp = 0
                #fn = 0
                
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
                
        kpis_phase_2_cd_0['acc'].append(accuracy)
        kpis_phase_2_cd_0['prec'].append(precision)
        kpis_phase_2_cd_0['spec'].append(specifity)
        kpis_phase_2_cd_0['sen'].append(sensitivity)

In [295]:
df_kpis_phase_2_cd_0 = pd.DataFrame(kpis_phase_2_cd_0)

In [296]:
df_kpis_phase_2_cd_0.describe()

,acc,prec,spec,sen
count,226.000000,226.000000,226.000000,226.000000
mean,55.815082,11.429434,94.074787,66.746086
std,25.899976,24.215666,11.706891,32.390697
min,2.338129,1.408451,0.000000,7.692308
25%,37.252698,2.711864,93.779904,34.615385
50%,55.665468,3.611111,97.017578,76.923077
75%,76.506295,4.790886,98.839387,100.000000
max,98.561151,100.000000,99.816176,100.000000



### Concept Drift Event Art 1


kpis_phase_2_cd_1 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}

for col in df_res_phase_2_cd_1.columns:
    if col != 'ano_labels' and col !='binary_label' and col !='cd_label':
        cm = confusion_matrix(lab_res_phase_2_cd_1, df_res_phase_2_cd_1[col])
        try:
            tn, fp, fn, tp  = cm.ravel()
        except:
            print('Cant ravel CM : {}'.format(col))
            #if df_results_phase_2_anos_3[col].all() == 1:
             #   tp = cm[0][0]
              #  tn = 0
               # fp = 0
                #fn = 0
                
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
                
        kpis_phase_2_cd_1['acc'].append(accuracy)
        kpis_phase_2_cd_1['prec'].append(precision)
        kpis_phase_2_cd_1['spec'].append(specifity)
        kpis_phase_2_cd_1['sen'].append(sensitivity)

In [299]:
df_kpis_phase_2_cd_1 = pd.DataFrame(kpis_phase_2_cd_1)

In [300]:
df_kpis_phase_2_cd_1.describe()

,acc,prec,spec,sen
count,2.260000e+02,2.260000e+02,226.0,2.260000e+02
mean,3.512133e-01,3.512133e-01,0.0,1.000000e+02
std,5.563437e-17,5.563437e-17,0.0,1.424240e-14
min,3.512133e-01,3.512133e-01,0.0,1.000000e+02
25%,3.512133e-01,3.512133e-01,0.0,1.000000e+02
50%,3.512133e-01,3.512133e-01,0.0,1.000000e+02
75%,3.512133e-01,3.512133e-01,0.0,1.000000e+02
max,3.512133e-01,3.512133e-01,0.0,1.000000e+02


### Concept Drift Event Art 2

In [301]:
kpis_phase_2_cd_2 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}

for col in df_res_phase_2_cd_2.columns:
    if col != 'ano_labels' and col !='binary_label' and col !='cd_label':
        cm = confusion_matrix(lab_res_phase_2_cd_2, df_res_phase_2_cd_2[col])
        try:
            tn, fp, fn, tp  = cm.ravel()
        except:
            print('Cant ravel CM : {}'.format(col))
            #if df_results_phase_2_anos_3[col].all() == 1:
             #   tp = cm[0][0]
              #  tn = 0
               # fp = 0
                #fn = 0
                
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
                
        kpis_phase_2_cd_2['acc'].append(accuracy)
        kpis_phase_2_cd_2['prec'].append(precision)
        kpis_phase_2_cd_2['spec'].append(specifity)
        kpis_phase_2_cd_2['sen'].append(sensitivity)

In [302]:
df_kpis_phase_2_cd_2 = pd.DataFrame(kpis_phase_2_cd_2)

In [303]:
df_kpis_phase_2_cd_2.describe()

,acc,prec,spec,sen
count,226.000000,226.000000,226.000000,226.000000
mean,62.595936,33.919049,76.507606,99.559208
std,24.182432,23.678011,14.847702,0.473826
min,11.761411,11.761411,0.000000,97.285714
25%,45.611873,17.767756,74.241766,99.428571
50%,64.362924,24.754257,81.813438,99.666667
75%,82.973957,40.795148,85.935896,99.857143
max,98.846262,91.571554,88.396001,100.000000


### Concept Drift Event Art 3

In [304]:
kpis_phase_2_cd_3 = {
    'acc': [],
    'prec': [],
    'spec': [],
    'sen': []
}

for col in df_res_phase_2_cd_3.columns:
    if col != 'ano_labels' and col !='binary_label' and col !='cd_label':
        cm = confusion_matrix(lab_res_phase_2_cd_3, df_res_phase_2_cd_3[col])
        try:
            tn, fp, fn, tp  = cm.ravel()
        except:
            print('Cant ravel CM : {}'.format(col))
            #if df_results_phase_2_anos_3[col].all() == 1:
             #   tp = cm[0][0]
              #  tn = 0
               # fp = 0
                #fn = 0
                
        accuracy, precision, specifity, sensitivity, f1_score = calc_cm_metrics(tp, tn, fp, fn)
                
        kpis_phase_2_cd_3['acc'].append(accuracy)
        kpis_phase_2_cd_3['prec'].append(precision)
        kpis_phase_2_cd_3['spec'].append(specifity)
        kpis_phase_2_cd_3['sen'].append(sensitivity)

In [305]:
df_kpis_phase_2_cd_3 = pd.DataFrame(kpis_phase_2_cd_3)

In [306]:
df_kpis_phase_2_cd_3.describe()

,acc,prec,spec,sen
count,226.000000,226.000000,226.000000,226.000000
mean,60.097597,17.839796,92.553231,94.059735
std,27.437492,26.552282,11.113984,7.299386
min,2.781856,2.781856,0.000000,62.777778
25%,42.514102,4.520232,93.663116,92.500000
50%,59.624449,6.316799,95.721666,96.805556
75%,82.358396,12.712794,96.876618,98.055556
max,99.791361,100.000000,98.235340,100.000000
